# Introduction
This 

In [1]:
import numpy as np


In [2]:
class Network:
    
    def __init__(self, architecture, activation= 'sigmoid'):
        self.architecture= architecture
        self.activation= activation
        self.parameters=[]
        self.buffered_op={}
        
        if type(self.activation) != str:
            assert len(self.activation) + 1 == len(self.architecture)
        for i in range(len(architecture) -1):
            layer_parameters= self.create_weights_and_biases(architecture[i], architecture[i + 1])
            self.parameters.append(layer_parameters)
        
        
    def create_weights_and_biases(self, in_nodes, out_nodes):
        std= np.sqrt(2.0/(in_nodes + out_nodes))
        w= np.random.normal(loc= 0, scale= std, size= (in_nodes, out_nodes))
        b= np.zeros((1, out_nodes))
        return {'weight':w , 'bias':b}
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return 0 if x<0 else x
    
    def forward_pass(self, x):
        
        for i, parameter in enumerate(self.parameter):
            w= parameter['weight']
            b= parameter['bias']
            if type(self.activation) != str:
                activation= self.activation[i]
            
            if not callable(activation):
                if lower(activation)=='sigmoid':
                    activation= self.sigmoid
                elif lower(activation) == 'relu':
                    activation= self.relu
                else:
                    raise Exception("Sorry, only relu and sigmoid supported currently") 
            
            h= np.matmul(x, w) + b
            a= activation(h)
            self.buffered_op['h' + str(i)]= h
            self.buffered_op['b' + str(i)]= a
        return a
            
        